

> Import libraries to use



In [21]:
import numpy as np

>  # Introduction to numpy (Skip if you already are familiar)

>> Creating a 1D array

In [22]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


>> Creating a 2D array


In [23]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


>> Creating an array full of zeros


In [24]:
a = np.zeros(shape=(10))
print(a)
a = np.zeros(shape=(5,2))
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


>> Infinity in numpy

In [25]:
print(np.inf)

inf


>> Max and Argmax

In [26]:
a = np.array([2,1,4,3])
print(np.max(a))
print(np.argmax(a))

4
2


>> From list to Numpy

In [27]:
l = [1,2,3,4]
print(l)
print(np.asarray(l))

[1, 2, 3, 4]
[1 2 3 4]


>> Random in numpy

In [28]:
# Array of Random integers ranging from 1 to 10 (with any size you want)
a = np.random.randint(low=1, high=10, size=(5,2))
print(a)

# Array of random elements of a list with any size you want
a = np.random.choice([0,1,2], size=(2,))

[[8 2]
 [4 7]
 [8 8]
 [5 5]
 [9 7]]


>> Shapes in numpy

In [29]:
a = np.random.randint(low=1, high=5, size=(4,2))
print(a.shape)
print(a)

# Reshape a to a vector of shape = (8,1)
a = a.reshape((8,1))
print(a.shape)
print(a)

(4, 2)
[[2 3]
 [4 4]
 [4 2]
 [2 2]]
(8, 1)
[[2]
 [3]
 [4]
 [4]
 [4]
 [2]
 [2]
 [2]]


# Pre-defined utilities

In [30]:

int_to_char = {
    0 : 'u',
    1 : 'r',
    2 : 'd',
    3 : 'l'
}

policy_one_step_look_ahead = {
    0 : [-1,0],
    1 : [0,1],
    2 : [1,0],
    3 : [0,-1]
}

def policy_int_to_char(pi,n):

    pi_char = ['']

    for i in range(n):
        for j in range(n):

            if i == 0 and j == 0 or i == n-1 and j == n-1:

                continue

            pi_char.append(int_to_char[pi[i,j]])

    pi_char.append('')

    return np.asarray(pi_char).reshape(n,n)

# 1- Policy evaluation

In [31]:
def policy_evaluation(n,pi,v,Gamma,threshhold):
  """
    This function should return the value function that follows the policy pi.
    Use the stopping criteria given in the problem statement.
  """
  while True:
    delta = 0.0 # Initialize the maximum change in value function
    v_new = np.copy(v)

    for i in range(n):
      for j in range(n):
        
        # Skip terminal states
        if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
          v_new[i, j] = 0.0
          continue

        action = pi[i, j]
        
        move = policy_one_step_look_ahead[action]
        next_i, next_j = i + move[0], j + move[1]

        # Stay in the same state if we leave the grid
        if next_i < 0 or next_i >= n or next_j < 0 or next_j >= n:
          next_i, next_j = i, j 

        reward = -1

        # Calculate the new value with the Bellman equation
        new_val = reward + Gamma * v[next_i, next_j]
        
        delta = max(delta, np.abs(new_val - v[i, j]))

        v_new[i, j] = new_val
    
    v = v_new

    if delta < threshhold:
      break
      
  return v

# 2- Policy improvement

In [32]:
def policy_improvement(n,pi,v,Gamma):
  """
    This function should return the new policy by acting in a greedy manner.
    The function should return as well a flag indicating if the output policy
    is the same as the input policy.

    Example:
      return new_pi, True if new_pi = pi for all states
      else return new_pi, False
  """
  pi_stable = True 
  new_pi = np.copy(pi)

  for i in range(n):
    for j in range(n):
      
      # Skip terminal states
      if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
        continue

      old_action = pi[i, j]
      action_values = []  

      for action in range(4): 
        move = policy_one_step_look_ahead[action]
        next_i, next_j = i + move[0], j + move[1]

        # Stay in the same state if we leave the grid
        if next_i < 0 or next_i >= n or next_j < 0 or next_j >= n:
          next_i, next_j = i, j
        
        reward = -1
        
        # Calculate the value for this action with the Bellman equation
        val = reward + Gamma * v[next_i, next_j]
        action_values.append(val)
      
      # Use the greedy policy
      best_action = np.argmax(action_values)

      new_pi[i, j] = best_action

      if old_action != best_action:
        pi_stable = False

  return new_pi, pi_stable

# 3- Policy Initialization

In [33]:
def policy_initialization(n):
  """
    This function should return the initial random policy for all states.
  """
  policy = np.random.randint(low=0, high=4, size=(n, n))
  return policy

In [34]:
policy_initialization(4)

array([[2, 3, 2, 1],
       [1, 1, 1, 0],
       [3, 0, 1, 0],
       [0, 2, 2, 1]], dtype=int32)

# 4- Policy Iteration algorithm

In [35]:
def policy_iteration(n,Gamma,threshhold):

    pi = policy_initialization(n=n)

    v = np.zeros(shape=(n,n))

    while True:

        v = policy_evaluation(n=n,v=v,pi=pi,threshhold=threshhold,Gamma=Gamma)

        pi , pi_stable = policy_improvement(n=n,pi=pi,v=v,Gamma=Gamma)

        if pi_stable:

            break

    return pi , v

# Main Code to Test

In [36]:
n = 4

Gamma = [0.8,0.9]

threshhold = 1e-4

for _gamma in Gamma:

    pi , v = policy_iteration(n=n,Gamma=_gamma,threshhold=threshhold)

    pi_char = policy_int_to_char(n=n,pi=pi)

    print()
    print("Gamma = ",_gamma)

    print()

    print(pi_char)

    print()
    print()

    print(v)



Gamma =  0.8

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0.   -1.   -1.8  -2.44]
 [-1.   -1.8  -2.44 -1.8 ]
 [-1.8  -2.44 -1.8  -1.  ]
 [-2.44 -1.8  -1.    0.  ]]

Gamma =  0.9

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]


Policy Iteration fails for Gamma = 1 because *policy_evaluation* enters an infinite loop if the policy has a cycle (like staying in place), as the value calculation never converges.

# Q1 Value iteration

In [37]:
def value_iteration(n, Gamma, threshhold):
  """
    This function implements the Value Iteration algorithm.
    It returns the optimal policy and the optimal value function.
  """

  v = np.zeros(shape=(n, n))
  
  while True:
    delta = 0.0
    v_new = np.copy(v) 

    for i in range(n):
      for j in range(n):
        
        if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
          v_new[i, j] = 0.0
          continue
          
        action_values = []  # Pour stocker les valeurs des 4 actions possibles
        
        for action in range(4):
          move = policy_one_step_look_ahead[action]
          next_i, next_j = i + move[0], j + move[1]
          
          if next_i < 0 or next_i >= n or next_j < 0 or next_j >= n:
            next_i, next_j = i, j
            
          reward = -1

          q_value = reward + Gamma * v[next_i, next_j]
          action_values.append(q_value)
          
        best_value = np.max(action_values)
        
        delta = max(delta, np.abs(best_value - v[i, j]))

        v_new[i, j] = best_value

    v = v_new

    if delta < threshhold:
      break
  
  pi = np.zeros(shape=(n, n), dtype=int)  
  
  for i in range(n):
    for j in range(n):
      if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
        continue  
        
      action_values = []
      for action in range(4):
        move = policy_one_step_look_ahead[action]
        next_i, next_j = i + move[0], j + move[1]
        
        if next_i < 0 or next_i >= n or next_j < 0 or next_j >= n:
          next_i, next_j = i, j
          
        reward = -1
        q_value = reward + Gamma * v[next_i, next_j]
        action_values.append(q_value)
        
      best_action = np.argmax(action_values)
      pi[i, j] = best_action
      
  return pi, v

In [38]:
n = 4

Gamma = [0.8, 0.9, 1.0] 

threshhold = 1e-4

for _gamma in Gamma:

    pi , v = value_iteration(n=n, Gamma=_gamma, threshhold=threshhold)

    pi_char = policy_int_to_char(n=n, pi=pi)

    print()
    print("Gamma = ",_gamma)

    print()

    print(pi_char)

    print()
    print()

    print(v)



Gamma =  0.8

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0.   -1.   -1.8  -2.44]
 [-1.   -1.8  -2.44 -1.8 ]
 [-1.8  -2.44 -1.8  -1.  ]
 [-2.44 -1.8  -1.    0.  ]]

Gamma =  0.9

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0.   -1.   -1.9  -2.71]
 [-1.   -1.9  -2.71 -1.9 ]
 [-1.9  -2.71 -1.9  -1.  ]
 [-2.71 -1.9  -1.    0.  ]]

Gamma =  1.0

[['' 'l' 'l' 'd']
 ['u' 'u' 'u' 'd']
 ['u' 'u' 'r' 'd']
 ['u' 'r' 'r' '']]


[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]


With the new function *value_iteration*, the situation Gamma = 1 works because it avoids that infinite loop. It skips *policy_evaluation* entirely by merging *policy_improvement*'s max operator into its main update loop, allowing it to converge by always picking the best action.